In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD

In [ ]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 'CPF',
        'CPN', 'DTAC', 'EGCO', 'HMPRO', 'INTUCH', 
        'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 
        'PTT', 'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [ ]:
lr = 0.1
epochs = 10000
batch_size = 100
optimizer = SGD(lr=lr)
model_size = 9
activator = 'tanh'
loss_method = 'mse'
metrics = ['mae', 'accuracy']

In [ ]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('./Data set/FIXED_SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        temp_value = 0
        for row in file_data:
            if row[1] is '':
                continue
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = float(row[1])
                if temp_value != 0:
                    unrealize = (temp - temp_value)/ temp_value
                else:
                    unrealize = 0
                unrealize = ["{0:.2f}".format(unrealize)]
                stock_data.append(unrealize)
                temp_value = temp
                
    return stock_data

In [ ]:
stock_datas = [ load_stock_data(x, 2008, 2017) for x in SET50 ]

min_count = len(stock_datas[0])
rounds = min_count - 30
train_index = int(rounds * 0.7)

In [ ]:
def create_model(conv_count, fully_count):
    i = 0
    model = Sequential()
    while(i <= conv_count):
        if i == 0:
            model.add(Conv2D(64, (7, 1), input_shape=(len(stock_datas), 30, 1), padding='same'))
        else:
            model.add(Conv2D(32, (3, 1)))
            
        model.add(Activation('tanh'))
        model.add(MaxPooling2D(pool_size = (2, 1), data_format= "channels_last"))
        i += 1
        
    i = 0
    model.add(Flatten())
    fisrt_dense = 1024
    while(i <= fully_count):
        if i == fully_count:
            model.add(Dropout(0.2))
            model.add(Dense(len(stock_datas)))
        else:
            model.add(Dense(fisrt_dense))  
            model.add(Activation('tanh'))
            fisrt_dense = fisrt_dense // 4
        i += 1
    
    return model

In [ ]:
models = [create_model(i/model_size, i%model_size) for i in range(model_size)]
for model in models:
    model.compile(loss=loss_method, optimizer=optimizer, metrics=metrics)

In [ ]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append(temp)
    index += 1
    
x = np.asarray(data)
x_train = x[0: train_index].astype('float32')
x_test = x[train_index: train_index + 1].astype('float32')

print(x_train.shape, x_test.shape)

In [ ]:
target = []
index = 0
while index < rounds:
    temp = [stock_datas[i][index + 30][0] for i in range(len(stock_datas))]
    target.append(temp)
    index += 1
    
y = np.asarray(target)
y_train = y[0: train_index].astype('float32')
y_test = y[train_index: rounds].astype('float32')

print(y_train.shape, y_test.shape)

In [ ]:
for index, model in enumerate(models):
    model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, verbose= 1)
    model_name = 'CNN' + str(index / 3) + 'FC' + str(index % 3) + '.h5'
    model.save(model_name)